In [11]:

import sympy as sp
from sympy import eye
from sympy import symbols, Matrix
# Define the symbols


# Compute the fourth order energy correction for the ground state |0>

def energy_sec(N):
    E2_N = 0  # Initialize the fourth order correction
    EN = ENN[N]
    dim = len(ENN)
    for m in range(dim):
        if m != N:
            VNm = V_ij[m, N]
            if sp.im(VNm) != 0 and sp.re(VNm) == 0:  # Pure imaginary
                VNm = sp.im(VNm)
            E2_N = E2_N + (VNm**2/(EN - ENN[m])) 
    return E2_N

def energy_four(N):
    E4_N = 0  # Initialize the fourth order correction
    EN = ENN[N]
    dim = len(ENN)
    for m in range(dim):
        if m != N and V_ij[N, m] !=0 :
            for p in range(dim):
                if p != N and V_ij[m, p]!=0 :
                    for l in range(dim):
                        if l != N and V_ij[p, l]!=0 and V_ij[l, N]!=0:
                            # Contribution from the main term
                            E4_N += ((V_ij[N, m] * V_ij[m, p] * V_ij[p, l] * V_ij[l, N] /
                                     ((EN - ENN[m]) * (EN - ENN[p]) * (EN - ENN[l]))))

    for m in range(dim):
        if m != N:
            for p in range(dim):
                if p != N:
                    VNm = V_ij[N, m]
                    VNp = V_ij[N, p]
                    if VNm == 0 or VNp ==0:
                        continue
                    if sp.im(VNm) != 0 and sp.re(VNm) == 0:  # Pure imaginary
                        VNm = sp.im(VNm)
                    if sp.im(VNp) != 0 and sp.re(VNp) == 0:  # Pure imaginary    
                        VNp = sp.im(VNp)
                    E4_N = E4_N - ((VNm**2/(EN - ENN[m])**2) * (VNp**2/(EN - ENN[p])))
    return E4_N


In [12]:
import sympy as sp

def state_first_order(N):
    """
    Computes the first-order correction to the N-th unperturbed state
    in standard non-degenerate perturbation theory.
    """
    dim = len(ENN)
    psi_corr_1 = [0]*dim
    E_n0 = ENN[N]

    for m in range(dim):
        if m != N:
            VNm = V_ij[m, N]
            if sp.im(VNm) != 0 and sp.re(VNm) == 0:
                VNm = sp.im(VNm)
            denominator = E_n0 - ENN[m]
            psi_corr_1[m] = VNm / denominator
        else:
            psi_corr_1[m] = 0
    return psi_corr_1


def state_second_order(N):
    """
    Computes the second-order correction to the N-th unperturbed state
    in standard non-degenerate perturbation theory, *including* the
    usual normalization piece along the unperturbed state |n^(0)>.
    """
    # --- 1) Get the first-order wavefunction ---
    psi_corr_1 = state_first_order(N)
    
    dim = len(ENN)
    psi_corr_2 = [0]*dim
    E_n0 = ENN[N]

    # --- 2) Compute the "bare" second-order part ---
    #     (psi_n^(2))_m = <m| V | psi_n^(1)> / [E_n^(0) - E_m^(0)], for m != n
    for m in range(dim):
        if m != N:
            sum_m = 0
            for k in range(dim):
                Vmk = V_ij[m, k]
                if sp.im(Vmk) != 0 and sp.re(Vmk) == 0:
                    Vmk = sp.im(Vmk)
                sum_m += Vmk * psi_corr_1[k]
            psi_corr_2[m] = sum_m / (E_n0 - ENN[m])
        else:
            psi_corr_2[m] = 0

    # --- 3) Add the normalization term along |n^(0)> ---
    #  The standard formula includes a piece:  - (1/2) |n^(0)> * SUM_{k != n} [ |V_{k,n}|^2 / (E_n0 - E_k0)^2 ]
    #  We'll compute that sum here and put it into psi_corr_2[N].
    norm_sum = 0
    for k in range(dim):
        if k != N:
            Vkn = V_ij[k, N]
            # If purely imaginary, keep the real coefficient
            if sp.im(Vkn) != 0 and sp.re(Vkn) == 0:
                Vkn = sp.im(Vkn)
            # We'll treat 'Vkn' potentially as complex and do |Vkn|^2:
            norm_sum += (Vkn * sp.conjugate(Vkn)) / (E_n0 - ENN[k])**2

    # This gets multiplied by -1/2, and placed in the coefficient for m = n
    psi_corr_2[N] = -0.5 * norm_sum

    return psi_corr_2



import sympy as sp

def state_first_order(N):
    """
    First-order wavefunction correction with no extra normalization piece.
    """
    dim = len(ENN)
    psi_corr_1 = [0]*dim
    E_n0 = ENN[N]

    for m in range(dim):
        if m != N:
            VNm = V_ij[m, N]
            if sp.im(VNm) != 0 and sp.re(VNm) == 0:
                VNm = sp.im(VNm)
            denominator = E_n0 - ENN[m]
            psi_corr_1[m] = VNm / denominator
        else:
            psi_corr_1[m] = 0
    return psi_corr_1

def state_second_order(N):
    """
    Second-order wavefunction correction *including* the usual
    'normalization' piece along |n^(0)>.
    """
    psi_corr_1 = state_first_order(N)
    dim = len(ENN)
    psi_corr_2 = [0]*dim
    E_n0 = ENN[N]

    # --- "Bare" part ---
    for m in range(dim):
        if m != N:
            sum_m = 0
            for k in range(dim):
                Vmk = V_ij[m, k]
                if sp.im(Vmk) != 0 and sp.re(Vmk) == 0:
                    Vmk = sp.im(Vmk)
                sum_m += Vmk * psi_corr_1[k]
            psi_corr_2[m] = sum_m / (E_n0 - ENN[m])
        else:
            psi_corr_2[m] = 0

    # --- Normalization piece: - 1/2 sum_{k != n} |V_{k,n}|^2 / (E_n0 - E_k0)^2
    norm_sum = 0
    for k in range(dim):
        if k != N:
            Vkn = V_ij[k, N]
            if sp.im(Vkn) != 0 and sp.re(Vkn) == 0:
                Vkn = sp.im(Vkn)
            norm_sum += (Vkn * sp.conjugate(Vkn)) / (E_n0 - ENN[k])**2

    psi_corr_2[N] = - sp.Rational(1, 2) * norm_sum

    return psi_corr_2

import sympy as sp

def state_third_order(N):
    """
    Hard-coded third-order wavefunction correction |n^(3)|, simplified:

    1) The triple-sum term:
       - sum_{k2!=n, k3!=n} [ V_{k1,k2} * V_{k2,k3} * V_{k3,n}
                             / ( (E_{k1}-E_n)*(E_n-E_{k2})*(E_n-E_{k3}) ) ]
    2) The single-sum term you noted:
       + sum_{k2!=n} [ |V_{n,k2}|^2 * V_{k1,n} / ( E_{k1,n} * E_{n,k2} )
                      * ( 1/E_{n,k1} + 1/(2 * E_{n,k2}) ) ]

    We set (psi_n^(3))_n = 0 (no diagonal piece / normalization piece).
    We also assume v_nn = 0 is ignored.
    """
    dim = len(ENN)
    psi_corr_3 = [0]*dim
    E_n0 = ENN[N]

    # Helper for denominators: E(a,b) = E_a^(0) - E_b^(0)
    def E_nk(a, b):
        return ENN[a] - ENN[b]

    for k1 in range(dim):
        if k1 == N:
            continue  # skip diagonal for the "bare" portion

        bracket_k1 = 0

        # -------------------------------------------------------
        # 1) The triple-sum term:
        #    - sum_{k2!=n, k3!=n} [ V_{k1,k2} V_{k2,k3} V_{k3,n}
        #      / ( (E_{k1}-E_n)*(E_n-E_{k2})*(E_n-E_{k3}) ) ]
        # -------------------------------------------------------
        for k2 in range(dim):
            if k2 == N:
                continue
            for k3 in range(dim):
                if k3 == N:
                    continue

                V_k1k2 = V_ij[k1, k2]
                V_k2k3 = V_ij[k2, k3]
                V_k3n  = V_ij[k3, N]

                # If purely imaginary in Sympy, extract the real-imag part
                if sp.im(V_k1k2) != 0 and sp.re(V_k1k2) == 0:
                    V_k1k2 = sp.im(V_k1k2)
                if sp.im(V_k2k3) != 0 and sp.re(V_k2k3) == 0:
                    V_k2k3 = sp.im(V_k2k3)
                if sp.im(V_k3n)  != 0 and sp.re(V_k3n)  == 0:
                    V_k3n  = sp.im(V_k3n)

                denom_triple = ( E_nk(k1, N) *  # (E_{k1}-E_n)
                                 E_nk(N, k2) *  # (E_n - E_{k2})
                                 E_nk(N, k3) ) # (E_n - E_{k3})

                term_triple = - (V_k1k2 * V_k2k3 * V_k3n) / denom_triple
                bracket_k1 += term_triple

        # -------------------------------------------------------
        # 2) The single-sum term:
        #    + sum_{k2!=n} [ |V_{n,k2}|^2 * V_{k1,n} / ( E_{k1,n} * E_{n,k2} )
        #                     * ( 1/E_{k1,n} + 1/(2 E_{n,k2}) ) ]
        # -------------------------------------------------------
        for k2 in range(dim):
            if k2 == N:
                continue

            # |V_{n,k2}|^2
            V_nk2 = V_ij[N, k2]
            if sp.im(V_nk2) != 0 and sp.re(V_nk2) == 0:
                V_nk2 = sp.im(V_nk2)
            square_nk2 = V_nk2 * sp.conjugate(V_nk2)

            V_k1n = V_ij[k1, N]
            if sp.im(V_k1n) != 0 and sp.re(V_k1n) == 0:
                V_k1n = sp.im(V_k1n)

            ek1n  = E_nk(k1, N)   # (E_{k1} - E_n)
            enk2  = E_nk(N, k2)   # (E_n - E_{k2})

            # ( 1/ek1n + 1/(2 * enk2) )
            factor = (1/ek1n) 

            denom_single = ek1n * enk2  # E_{k1,n} * E_{n,k2}
            term_single = ( square_nk2 * V_k1n / denom_single ) * factor

            bracket_k1 += term_single

        # Finally store sum of both terms in psi_corr_3
        psi_corr_3[k1] = bracket_k1

    # The coefficient along |n^(0)| is set to zero
    psi_corr_3[N] = 0

    return psi_corr_3

# def energy_fourth_order(N):
#     psi_corr_3 = state_third_order(N)  # fully orthogonal 
#     dim = len(ENN)  # number of basis states
#     psi0 = [0]*dim
#     psi0[N] = 1
#     # compute val = <n^(0)| V | n^(3)>
#     val = 0
#     for i in range(dim):
#         for j in range(dim):
#             val += sp.conjugate(psi0[i]) * V_ij[i,j] * psi_corr_3[j]
#     E4 = val 
#     return E4


In [13]:
from sympy.physics.quantum import TensorProduct
from sympy import symbols, sqrt
nc=5
ac = sp.zeros(nc)
acd = sp.zeros(nc)

# Fill the matrices using sympy functions
for i in range(nc-1):
    ac[i, i+1] = sqrt(i+1)
    acd[i+1, i] = sqrt(i+1)


In [14]:
omega1, omega2 , omegac,  = symbols('omega_q omega_2 omega_a', real=True)
alpha1, alpha2 , alpha3,  = symbols('alpha_q alpha_2 K_a', real=True)
delta = symbols('delta',real = True)
chi12, chi2c, chi1c = symbols('chi_12 chi_2c chi', real=True)
A = symbols('A', real=True)

n1 = 5
n2 = 5

In [15]:
Vc = A/2*(ac+acd)
V_ij = TensorProduct(Vc, sp.eye(n2))

ENN = [
    i*omega1 + l*delta + i*(i-1) * alpha1 / 2 + l*(l-1) * alpha3 / 2 + chi1c*i*l
    for l in range(nc)
    for i in range(n1)
]


In [16]:
def create_order_matrix(state_func, n_states):
    rows = []
    for i in range(n_states):
        # state_func(i) should return an array (list) of sympy expressions
        rows.append(state_func(i))
    return sp.Matrix(rows)

# Simplify creation of these TensorProducts
a = TensorProduct(ac, sp.eye(n2))
ad = TensorProduct(acd, sp.eye(n2))
ada = ad * a  # Example operation
b = TensorProduct(sp.eye(n1), ac)
bd = TensorProduct(sp.eye(n1), acd)
# Create the matrices
n_states = n1 * n2
first_order  = create_order_matrix(state_first_order,  n_states)
# second_order = create_order_matrix(state_second_order, n_states)
# third_order  = create_order_matrix(state_third_order,  n_states)

In [17]:
g = sp.Symbol('g', real=True)
delta = sp.Symbol('Delta', real=True)

In [18]:
energy_sec(1) - energy_sec(0)

A**2/(4*(-chi - delta)) + A**2/(4*delta)

In [19]:
energy_four(1) - energy_four(0)

A**4/(8*(-chi - delta)**2*(-K_a - 2*chi - 2*delta)) - A**4/(16*(-chi - delta)**3) - A**4/(8*delta**2*(-K_a - 2*delta)) - A**4/(16*delta**3)

In [8]:

U = sp.eye(n_states) + first_order 
Ud = U.T
expr = (Ud @ (ad+g/delta*bd)@(a+g/delta*b) @ U)

In [9]:
expr

Matrix([
[                    A**2/(4*delta**2),                                                                                         A*g/(2*Delta*delta),                                                                                                               0,                                                                                                                   0,                                                                                                                 0,                                                 A/(2*delta),                                                                                                                                              A**2*g/(4*Delta*delta*(-chi - delta)),                                                                                                                                                                                          0,                                                                     

In [10]:
(Ud @ (ad)@(a) @ U)

Matrix([
[                    A**2/(4*delta**2),                                                   0,                                                       0,                                                       0,                                                       0,                                            A/(2*delta),                                                                  0,                                                                      0,                                                                      0,                                                                      0,                          sqrt(2)*A**2/(4*delta*(-K_a - delta)),                                                                          0,                                                                              0,                                                                              0,                                                                              0,        

In [11]:

U = sp.eye(n_states) + first_order + second_order 
Ud = U.T

def extract_A4_terms(expr, A):
    """Return the sub-expression from expr that contains A^4."""
    factored_terms = [sp.factor(term) for term in expr.as_ordered_terms()]
    summed_expr = sum(factored_terms)
    
    matching_terms = []
    for term in summed_expr.as_ordered_terms():
        fac_term = sp.factor(term)
        # Check that this term has A^4 (degree 4 in A)
        if sp.poly(fac_term, A).degree() == 4:
            matching_terms.append(term)
    
    return sum(matching_terms)

expr = ((Ud @ ada @ U)[0, 10] ** 2).expand()
(extract_A4_terms(expr, A) ).factor()

NameError: name 'second_order' is not defined

In [12]:
from qutip import Qobj, qsave
expr = ((Ud @ ada @ U)[0, 10] ** 2)
# Save the Qobj
qsave(expr, 'ada')

In [13]:
expr = ((Ud@a@U)[5,0]**2).expand()
expr = extract_A4_terms(expr, A)
expr =  expr - expr.args[0] - expr.args[2]
# Save the Qobj
qsave(expr, 'a')


NameError: name 'extract_A4_terms' is not defined

In [33]:
expr

A**4/(4*(K_a + delta)**2*(K_a + 2*delta)**2)

In [64]:
energy_sec(1)

A**2/(4*(-chi - delta))

In [57]:
chi1c
delta

Delta

In [14]:
energy_four(1)

A**4/(8*(-chi - delta)**2*(-K_a - 2*chi - 2*delta)) - A**4/(16*(-chi - delta)**3)

In [15]:
E01 = (energy_sec(1)-energy_sec(0)).expand().simplify()

In [16]:
sp.latex(E01)

'\\frac{A^{2} \\chi}{4 \\delta \\left(\\chi + \\delta\\right)}'

In [17]:
E1 = energy_sec(1) + energy_four(1)

In [18]:
xx = (energy_four(1) - energy_four(0))

In [19]:
sp.latex((xx.args[2]+xx.args[3]).simplify().factor())

'\\frac{A^{4} \\chi \\left(K_{a} \\chi + 2 K_{a} \\delta + 2 \\chi^{2} + 6 \\chi \\delta + 6 \\delta^{2}\\right)}{8 \\delta^{2} \\left(K_{a} + 2 \\delta\\right) \\left(\\chi + \\delta\\right)^{2} \\left(K_{a} + 2 \\chi + 2 \\delta\\right)}'

In [20]:
energy_sec(6) - energy_sec(5) - energy_sec(1) + energy_sec(0)

A**2/(2*(-K_a - chi - delta)) + A**2/(4*(chi + delta)) - A**2/(4*(-chi - delta)) - A**2/(2*(-K_a - delta)) - A**2/(2*delta)

In [21]:
energy_four(6) - energy_four(5) - energy_four(1) + energy_four(0)

-A**4/(4*(-K_a - chi - delta)**3) + 3*A**4/(8*(-3*K_a - 2*chi - 2*delta)*(-K_a - chi - delta)**2) - A**4/(8*(chi + delta)*(-K_a - chi - delta)**2) - A**4/(8*(chi + delta)**2*(-K_a - chi - delta)) - A**4/(16*(chi + delta)**3) - A**4/(8*(-chi - delta)**2*(-K_a - 2*chi - 2*delta)) + A**4/(16*(-chi - delta)**3) + A**4/(4*(-K_a - delta)**3) - 3*A**4/(8*(-3*K_a - 2*delta)*(-K_a - delta)**2) + A**4/(8*delta*(-K_a - delta)**2) + A**4/(8*delta**2*(-K_a - delta)) + A**4/(8*delta**2*(-K_a - 2*delta)) + A**4/(8*delta**3)

In [12]:
import sympy as sp

# Assuming E01 is already defined
E01 = (energy_sec(1) - energy_sec(0)).expand().simplify()

# Convert the expression to LaTeX
latex_expr = sp.latex(E01)

# Replace double backslashes with single backslashes
latex_expr_single_backslash = latex_expr.replace('\\\\', '\\')

# Print the modified LaTeX expression
print(latex_expr_single_backslash)

\frac{A^{2} \chi}{4 \delta \left(\chi + \delta\right)}


In [ ]:
def convert_latex(expr):
    latex_expr = sp.latex(expr)
    return latex_expr.replace('\\\\', '\\')